# Agenda

- Generators (generator functions)
- Concurrency
    - Threads
    - Processes
    - `asyncio`
- Profiling    
- NumPy + Pandas

- Single-quoted string: `''`
- Double-quoted string: `""`
- `"""triple quoted string"""`
- `r'c:\a\b\c\d\e'`  # auto-doubles backslashes
- `b'abc'` # bytes, not characters
- `f'abc{x}'`  # replaces anything in `{}` with its value -- starting from Python 3.6

In [2]:
# 3.8 added trailing = to f-strings

x = 100
y = [10, 20, 30]

f'{x=}, {y=}'

'x=100, y=[10, 20, 30]'

In [5]:
def myfunc():
    return 1
    return 2
    return 3

In [6]:
myfunc()

1

In [7]:
import dis

In [8]:
dis.dis(myfunc)

  2           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


In [9]:
def myfunc():
    yield 1
    yield 2
    yield 3

In [10]:
myfunc() 

<generator object myfunc at 0x108f5ad50>

# Iterator protocol

1. We run `iter` on the object, we get the object's iterator back (or an exception).
2. We can run `next` on the returned object
    - 